In [2]:
from sage.all import *
import sys
sys.path.insert(0, '/mnt/d/Kuliah/ctf/research/kzg-snark')

from kzg import KZG
from fft_ff import fft_ff, fft_ff_interpolation
from encoder import Encoder
from transcript import Transcript

class Prover:
    """
    Marlin zkSNARK prover implementation.
    
    The prover generates a zero-knowledge proof for an R1CS instance using the optimized
    protocol described in Appendix E of the Marlin paper.
    """
    
    def __init__(self, curve_type="bn254"):
        """
        Initialize the Marlin prover with a KZG polynomial commitment scheme.
        
        Args:
            curve_type: Type of curve to use ('bn254' or 'bls12_381')
        """
        self.kzg = KZG(curve_type=curve_type)
        
    def prove(self, ipk, x, w, zero_knowledge_bound=2):
        """
        Generate a zero-knowledge proof for the R1CS instance.
        
        Args:
            ipk: Index proving key from the indexer
            x: Public input
            w: Witness
            zero_knowledge_bound: Parameter 'b' for zero-knowledge (default: 2)
            
        Returns:
            dict: The proof
        """
        # Extract data from index proving key
        ck = ipk["ck"]
        A, B, C = ipk["A"], ipk["B"], ipk["C"]
        indexer_polys = ipk["polynomials"]
        H, K = ipk["subgroups"]["H"], ipk["subgroups"]["K"]
        n, m = ipk["subgroups"]["n"], ipk["subgroups"]["m"]
        g_K = ipk["subgroups"]["g_K"]
        v_H, v_K = ipk["vanishing_polys"]["v_H"], ipk["vanishing_polys"]["v_K"]
        R = self.kzg.R
        X = self.kzg.X
        Fq = self.kzg.Fq
        
        # Create an encoder with the same field
        self.encoder = Encoder(self.kzg.curve_order)
        self.encoder.update_state(A, B, C)
        
        # Create a transcript for the Fiat-Shamir transform
        transcript = Transcript("marlin-proof", Fq)
        
        # Phase 1: Encode witness and linear combinations
        z = list(x) + list(w)  # Full variable assignment
        x_size = len(x)
        v_H_x = prod([X - h for h in H[:x_size]])
        v_H_w = prod([X - h for h in H[x_size:]])
        
        # Encode witness
        encoded_witness = self.encoder.encode_witness(z, x_size)
        
        # Encode linear combinations
        encoded_combinations = self.encoder.encode_linear_combinations(z)
        
        # Get the polynomials needed for the protocol
        w_poly = encoded_witness["w_poly"]
        x_poly = encoded_witness["x_poly"]
        zA_poly = encoded_combinations["zA_poly"]
        zB_poly = encoded_combinations["zB_poly"]
        zC_poly = encoded_combinations["zC_poly"]
        
        # Add randomness for zero-knowledge (bounded independence)
        b = zero_knowledge_bound
        w_random = sum(Fq.random_element() * X**i for i in range(b))
        zA_random = sum(Fq.random_element() * X**i for i in range(b))
        zB_random = sum(Fq.random_element() * X**i for i in range(b))
        zC_random = sum(Fq.random_element() * X**i for i in range(b))
        
        w_masked = w_poly + w_random * v_H_w
        zA_masked = zA_poly + zA_random * v_H
        zB_masked = zB_poly + zB_random * v_H
        zC_masked = zC_poly + zC_random * v_H
        z_masked = w_masked * v_H_x + x_poly
        
        # Compute h_0 such that zA_masked(X)·zB_masked(X) - zC_masked(X) = h_0(X)·v_H(X)
        h_0 = (zA_masked * zB_masked - zC_masked) // v_H
        assert h_0 * v_H == zA_masked * zB_masked - zC_masked, "h_0 is undefined"
        
        # Generate random polynomial s(X) such that sum(s(H)) = 0
        s_random = sum(Fq.random_element() * X**i for i in range(2*n+b-1))
        s_sum = sum(s_random(h) for h in H)
        s = s_random - s_sum/len(H)  # Adjust to ensure sum over H is zero
        
        # First round commitments
        first_round_polys = [w_masked, zA_masked, zB_masked, zC_masked, h_0, s]
        first_round_commitments = self.kzg.commit(ck, first_round_polys)
        
        # Add commitments to transcript
        transcript.append_message("round1-commitments", first_round_commitments)
        
        # Get first round challenges
        eta_A = transcript.get_challenge("eta_A")
        eta_B = transcript.get_challenge("eta_B")
        eta_C = transcript.get_challenge("eta_C")
        alpha = transcript.get_challenge("alpha")
        
        # Ensure alpha is not in H (as required by protocol)
        while alpha in H:
            alpha = transcript.get_challenge("alpha-retry")
        
        # Compute t(X) - the combined polynomial for r_M(α, X)
        t = self._compute_t_polynomial(
            indexer_polys, eta_A, eta_B, eta_C, alpha, v_H, K, R
        )
        
        # Compute first sumcheck polynomial
        r = lambda a, b: self.encoder.u_H(a, b)  # Helper function for u_H
        
        # Compute the sum of s(X) + r(α, X) * (Σ η_M * z_M(X)) - t(X) * z(X) over H
        poly = R(s + r(alpha, X)*(eta_A * zA_masked + eta_B * zB_masked + eta_C * zC_masked) - t * z_masked)
        
        # Divide by vanishing polynomial v_H to get h_1 and g_1
        h_1 = poly // v_H
        g_1 = poly % v_H

        assert g_1.constant_coefficient() == 0, "Sum over H is not 0"
        g_1 = g_1 // X
        assert h_1*v_H + X * g_1 == poly, "h_1 and g_1 is undefined"
        
        # Second round commitments
        second_round_polys = [t, g_1, h_1]
        second_round_commitments = self.kzg.commit(ck, second_round_polys)
        
        transcript.append_message("round2-commitments", second_round_commitments)
        
        # Get second round challenge
        beta_1 = transcript.get_challenge("beta_1")
        
        # Ensure beta_1 is not in H
        while beta_1 in H:
            beta_1 = transcript.get_challenge("beta_1-retry")
        
        # Calculate a(X) and b(X) polynomials for the third sumcheck
        a, b = self._compute_a_b_polynomials(
            indexer_polys, eta_A, eta_B, eta_C, beta_1, alpha, v_H, R
        )
        
        # Calculate sumcheck for the value of t(β₁)
        t_beta1 = t(beta_1)
        
        # Following the approach in Marlin paper page 29, we need to 
        # interpolate f_2 and then compute g_2 and h_2
        f_2 = self._compute_f2_polynomial(indexer_polys, eta_A, eta_B, eta_C, beta_1, alpha, v_H, v_K, K, g_K, Fq, R)
        assert f_2.constant_coefficient() == t_beta1 / m, "f_2 is wrong"

        g_2 = f_2 // X
        h_2 = (a - b*f_2) // v_K
        assert h_2 * v_K == a - b * (X * g_2 + t_beta1 / m), "h_2 and g_2 is undefined"
        
        # Third round commitments
        third_round_polys = [g_2, h_2]
        third_round_commitments = self.kzg.commit(ck, third_round_polys)
        
        transcript.append_message("round3-commitments", third_round_commitments)
        
        # Get third round challenge
        beta_2 = transcript.get_challenge("beta_2")
        
        # Polynomials for beta_1
        polys_beta1 = [w_masked, zA_masked, zB_masked, zC_masked, h_0, s, t, g_1, h_1]
        evals_beta1 = [p(beta_1) for p in polys_beta1]
        
        # Polynomials for beta_2
        polys_beta2 = [g_2, h_2] + indexer_polys
        evals_beta2 = [p(beta_2) for p in polys_beta2]
        
        # Add evaluations to transcript
        transcript.append_message("evaluations-beta1", evals_beta1)
        transcript.append_message("evaluations-beta2", evals_beta2)
        
        # Get opening challenge for batch verification
        xi_1 = transcript.get_challenge("xi_1")
        xi_2 = transcript.get_challenge("xi_2")
        
        # Generate KZG batch proofs
        proof_beta1 = self.kzg.open(ck, polys_beta1, beta_1, xi_1)
        proof_beta2 = self.kzg.open(ck, polys_beta2, beta_2, xi_2)
        
        # Assemble the final proof
        proof = {
            "commitments": {
                "first_round": first_round_commitments,
                "second_round": second_round_commitments,
                "third_round": third_round_commitments
            },
            "evaluations": {
                "beta1": evals_beta1,
                "beta2": evals_beta2
            },
            "kzg_proofs": {
                "beta1": proof_beta1,
                "beta2": proof_beta2
            }
        }
        
        return proof
        
        
    def _compute_t_polynomial(self, indexer_polys, eta_A, eta_B, eta_C, alpha, v_H, K, R):
        """
        Compute the t(X).
        
        This is the optimized version following Appendix E.
        
        Args:
            indexer_polys: List of polynomials from the indexer
            eta_A, eta_B, eta_C: Challenge coefficients for matrices
            alpha: Challenge point
            v_H: Vanishing polynomial for domain H
            K: Domain K
            R: Polynomial ring
            
        Returns:
            t_poly: The combined polynomial t(X)
        """
        # Extract row, col, val polynomials for each matrix
        row_A, col_A, val_A = indexer_polys[0:3]
        row_B, col_B, val_B = indexer_polys[3:6]
        row_C, col_C, val_C = indexer_polys[6:9]
        
        X = R.gen()
        t_poly = R(0)
        
        # Following the optimization in Appendix E, we compute:
        # t(X) = Σ_{M∈{A,B,C}} η_M · Σ_{κ∈K} v_H(X)v_H(α)·val_M*(κ) / ((X - row_M*(κ))(α - col_M*(κ)))
        
        for kappa in K:
            # Term for matrix A
            term_A = (v_H(X) * v_H(alpha) * val_A(kappa)) // ((X - row_A(kappa)) * (alpha - col_A(kappa)))
            t_poly += eta_A * term_A
            
            # Term for matrix B
            term_B = (v_H(X) * v_H(alpha) * val_B(kappa)) // ((X - row_B(kappa)) * (alpha - col_B(kappa)))
            t_poly += eta_B * term_B
            
            # Term for matrix C
            term_C = (v_H(X) * v_H(alpha) * val_C(kappa)) // ((X - row_C(kappa)) * (alpha - col_C(kappa)))
            t_poly += eta_C * term_C
        
        return t_poly
        
    def _compute_a_b_polynomials(self, indexer_polys, eta_A, eta_B, eta_C, beta_1, alpha, v_H, R):
        """
        Compute the a(X) and b(X) polynomials for the second sumcheck in the Marlin protocol.
        
        Args:
            indexer_polys: List of polynomials from the indexer
            eta_A, eta_B, eta_C: Challenge coefficients for matrices
            beta_1, alpha: Challenge points
            v_H: Vanishing polynomial for domain H
            R: Polynomial ring
            
        Returns:
            tuple: (a_poly, b_poly)
        """
        # Extract row, col, val polynomials for each matrix
        row_A, col_A, val_A = indexer_polys[0:3]
        row_B, col_B, val_B = indexer_polys[3:6]
        row_C, col_C, val_C = indexer_polys[6:9]
        a = R(0)
        b = R(1)  # Start with 1 for the product
        
        # Process each matrix
        for matrix_idx, (eta, row, col, val) in enumerate([
            (eta_A, row_A, col_A, val_A),
            (eta_B, row_B, col_B, val_B),
            (eta_C, row_C, col_C, val_C)
        ]):
            # Calculate the product term for other matrices
            other_product = R(1)
            for other_idx, (other_row, other_col) in enumerate([
                (row_A, col_A), (row_B, col_B), (row_C, col_C)
            ]):
                if other_idx != matrix_idx:
                    other_product *= (beta_1 - other_row) * (alpha - other_col)
            
            # Add term to a(X)
            a += eta * v_H(beta_1) * v_H(alpha) * val * other_product
            
            # Update b(X) with this matrix's factors
            b *= (beta_1 - row) * (alpha - col)
        
        return a, b

    def _compute_f2_polynomial(self, indexer_polys, eta_A, eta_B, eta_C, beta_1, alpha, v_H, v_K, K, g_K, Fq, R):
        """
        Compute the f2(X) polynomial for the second sumcheck in the Marlin protocol.
        
        Args:
            indexer_polys: List of polynomials from the indexer [row_A, col_A, val_A, row_B, col_B, val_B, row_C, col_C, val_C]
            eta_A, eta_B, eta_C: Challenge coefficients for matrices
            beta_1, alpha: Challenge points (neither in H)
            v_H: Vanishing polynomial for domain H
            v_K: Vanishing polynomial for domain K
            K: Domain K (list of field elements)
            g_K: Generator for domain K
            Fq: Finite field
            R: Polynomial ring
            
        Returns:
            f2_poly
        """
        # Extract row, col, val polynomials for each matrix
        row_A, col_A, val_A = indexer_polys[0:3]
        row_B, col_B, val_B = indexer_polys[3:6]
        row_C, col_C, val_C = indexer_polys[6:9]
        
        # Pre-compute v_H values
        v_H_beta1 = v_H(beta_1)
        v_H_alpha = v_H(alpha)
        
        # Pre-compute evaluations at all points in K for efficiency
        # This is crucial for performance as we need the values at each kappa ∈ K
        row_A_evals = fft_ff(list(row_A), g_K, Fq)
        col_A_evals = fft_ff(list(col_A), g_K, Fq)
        val_A_evals = fft_ff(list(val_A), g_K, Fq)
        
        row_B_evals = fft_ff(list(row_B), g_K, Fq)
        col_B_evals = fft_ff(list(col_B), g_K, Fq)
        val_B_evals = fft_ff(list(val_B), g_K, Fq)
        
        row_C_evals = fft_ff(list(row_C), g_K, Fq)
        col_C_evals = fft_ff(list(col_C), g_K, Fq)
        val_C_evals = fft_ff(list(val_C), g_K, Fq)
        
        # Compute f2(κ) for each κ ∈ K using the formula from Appendix E
        f2_evals = []
        
        for i in range(len(K)):
            # Calculate denominator terms for each matrix
            denom_A = (beta_1 - row_A_evals[i]) * (alpha - col_A_evals[i])
            denom_B = (beta_1 - row_B_evals[i]) * (alpha - col_B_evals[i])
            denom_C = (beta_1 - row_C_evals[i]) * (alpha - col_C_evals[i])
            
            # Calculate individual terms with appropriate safeguards for division by zero
            term_A = v_H_beta1 * v_H_alpha * val_A_evals[i] / denom_A if denom_A != 0 else 0
            term_B = v_H_beta1 * v_H_alpha * val_B_evals[i] / denom_B if denom_B != 0 else 0
            term_C = v_H_beta1 * v_H_alpha * val_C_evals[i] / denom_C if denom_C != 0 else 0
            
            # Combine with appropriate challenge values
            f2_evals.append(eta_A * term_A + eta_B * term_B + eta_C * term_C)
        
        # Interpolate to get the polynomial
        f_2 = fft_ff_interpolation(f2_evals, g_K, Fq)

        return f_2

In [5]:
import sys
from sage.all import prod

sys.path.insert(0, '/mnt/d/Kuliah/ctf/research/kzg-snark')
from kzg import KZG
from transcript import Transcript

class Verifier:
    """
    Marlin zkSNARK verifier implementation.
    
    The verifier checks the proof generated by the prover for an R1CS instance.
    It follows the optimized protocol described in Appendix E of the Marlin paper.
    """
    
    def __init__(self, curve_type="bn254"):
        """
        Initialize the Marlin verifier with a KZG polynomial commitment scheme.
        
        Args:
            curve_type: Type of curve to use ('bn254' or 'bls12_381')
        """
        self.kzg = KZG(curve_type=curve_type)
        
    def verify(self, ivk, x, proof):
        """
        Verify a Marlin proof for the given public input.
        
        Args:
            ivk: Index verification key from the indexer
            x: Public input
            proof: Proof generated by the prover
            
        Returns:
            bool: True if the proof is valid, False otherwise
        """
        # Extract data from verification key and proof
        rk = ivk["rk"]
        index_commitments = ivk["commitments"]
        n, m = ivk["subgroups"]["n"], ivk["subgroups"]["m"]
        g_H = ivk["subgroups"]["g_H"]
        
        # Extract proof components
        first_round_commitments = proof["commitments"]["first_round"]
        second_round_commitments = proof["commitments"]["second_round"]
        third_round_commitments = proof["commitments"]["third_round"]
        evals_beta1 = proof["evaluations"]["beta1"]
        evals_beta2 = proof["evaluations"]["beta2"]
        kzg_proof_beta1 = proof["kzg_proofs"]["beta1"]
        kzg_proof_beta2 = proof["kzg_proofs"]["beta2"]
        
        # Create a transcript for the Fiat-Shamir transform
        R = self.kzg.R
        Fq = self.kzg.Fq
        transcript = Transcript("marlin-proof", Fq)
        
        # Recreate the transcript to generate the same challenges as the prover
        transcript.append_message("round1-commitments", first_round_commitments)
        
        # Get first round challenges
        eta_A = transcript.get_challenge("eta_A")
        eta_B = transcript.get_challenge("eta_B")
        eta_C = transcript.get_challenge("eta_C")
        alpha = transcript.get_challenge("alpha")
        
        # Add second round commitments and get challenge
        transcript.append_message("round2-commitments", second_round_commitments)
        beta_1 = transcript.get_challenge("beta_1")
        
        # Add third round commitments and get challenge
        transcript.append_message("round3-commitments", third_round_commitments)
        beta_2 = transcript.get_challenge("beta_2")
        
        # Add evaluations to transcript
        transcript.append_message("evaluations-beta1", evals_beta1)
        transcript.append_message("evaluations-beta2", evals_beta2)
        
        # Get opening challenges for batch verification
        xi_1 = transcript.get_challenge("xi_1")
        xi_2 = transcript.get_challenge("xi_2")
        
        # Extract evaluations from the proof
        [w_beta1, zA_beta1, zB_beta1, zC_beta1, h0_beta1, s_beta1, 
         t_beta1, g1_beta1, h1_beta1] = evals_beta1
        
        # The first two elements in evals_beta2 are g2(β₂) and h2(β₂)
        g2_beta2 = evals_beta2[0]
        h2_beta2 = evals_beta2[1]
        
        # The remaining elements are the evaluations of the 9 indexer polynomials at β₂
        [row_A_beta2, col_A_beta2, val_A_beta2,
         row_B_beta2, col_B_beta2, val_B_beta2,
         row_C_beta2, col_C_beta2, val_C_beta2] = evals_beta2[2:11]
        
        # Get commitments
        [w_comm, zA_comm, zB_comm, zC_comm, h0_comm, s_comm] = first_round_commitments
        [t_comm, g1_comm, h1_comm] = second_round_commitments
        [g2_comm, h2_comm] = third_round_commitments
        
        # Verify polynomial relations
        # 1. Check entry-wise product: zA(β₁)·zB(β₁) - zC(β₁) = h₀(β₁)·v_H(β₁)
        v_H_beta1 = beta_1**n - 1  # v_H(β₁) = β₁^n - 1
        if zA_beta1 * zB_beta1 - zC_beta1 != h0_beta1 * v_H_beta1:
            print("❌ Entry-wise product check failed")
            return False
        
        # 2. Check first sumcheck: s(β₁) + r(α,β₁)(Σ η_M·z_M(β₁)) - t(β₁)·z(β₁) = h₁(β₁)·v_H(β₁) + β₁·g₁(β₁)
        # First compute r(α,β₁) which is u_H(α,β₁) = (α^n - β₁^n)/(α - β₁)
        r_alpha_beta1 = (alpha**n - beta_1**n) / (alpha - beta_1)
        
        H_x = [g_H**i for i in range(len(x))]
        # Compute z(β₁) from w(β₁)
        # For verification, we need to compute x_poly(β₁)
        # Since x_poly depends on the public input, we reconstruct it
        # First, we compute v_H_x (vanishing polynomial for H[:len(x)])
        v_H_x_beta1 = prod([(beta_1 - H_x[i]) for i in range(len(x))])
        
        # Compute x_poly(β₁) using the public input x
        # This is a simple approximation - in practice we would need to 
        # compute the Lagrange basis polynomials properly
        x_points = [(H_x[i], x[i]) for i in range(len(x))]
        x_poly = R.lagrange_polynomial(x_points)
        x_poly_beta1 = x_poly(beta_1)
        
        # Compute z(β₁) = w(β₁)·v_H_x(β₁) + x_poly(β₁)
        z_beta1 = w_beta1 * v_H_x_beta1 + x_poly_beta1
        
        # Check the first sumcheck relation
        sumcheck1 = s_beta1 + r_alpha_beta1 * (eta_A * zA_beta1 + eta_B * zB_beta1 + eta_C * zC_beta1) - t_beta1 * z_beta1
        expected_sumcheck1 = h1_beta1 * v_H_beta1 + beta_1 * g1_beta1
        
        if sumcheck1 != expected_sumcheck1:
            print("❌ First sumcheck relation failed")
            return False
        
        # 3. Check second sumcheck relation with evaluations at β₂
        v_K_beta2 = beta_2**m - 1  # v_K(β₂) = β₂^m - 1
        v_H_beta1 = beta_1**n - 1  # v_H(β₁) = β₁^n - 1
        v_H_alpha = alpha**n - 1   # v_H(α) = α^n - 1
        
        # Now we can compute a(β₂) and b(β₂) using the indexed polynomial evaluations
        
        # For a(β₂), we compute:
        # a(β₂) = Σ_{M∈{A,B,C}} η_M · v_H(β₁)·v_H(α)·val_M(β₂) · Π_{N≠M} (β₁-row_N(β₂))·(α-col_N(β₂))
        
        # Term for matrix A
        term_A_other_prod = ((beta_1 - row_B_beta2) * (alpha - col_B_beta2) * 
                             (beta_1 - row_C_beta2) * (alpha - col_C_beta2))
        term_A = eta_A * v_H_beta1 * v_H_alpha * val_A_beta2 * term_A_other_prod
        
        # Term for matrix B
        term_B_other_prod = ((beta_1 - row_A_beta2) * (alpha - col_A_beta2) * 
                             (beta_1 - row_C_beta2) * (alpha - col_C_beta2))
        term_B = eta_B * v_H_beta1 * v_H_alpha * val_B_beta2 * term_B_other_prod
        
        # Term for matrix C
        term_C_other_prod = ((beta_1 - row_A_beta2) * (alpha - col_A_beta2) * 
                             (beta_1 - row_B_beta2) * (alpha - col_B_beta2))
        term_C = eta_C * v_H_beta1 * v_H_alpha * val_C_beta2 * term_C_other_prod
        
        a_beta2 = term_A + term_B + term_C
        
        # For b(β₂), we compute:
        # b(β₂) = Π_{M∈{A,B,C}} (β₁-row_M(β₂))·(α-col_M(β₂))
        b_beta2 = ((beta_1 - row_A_beta2) * (alpha - col_A_beta2) * 
                   (beta_1 - row_B_beta2) * (alpha - col_B_beta2) * 
                   (beta_1 - row_C_beta2) * (alpha - col_C_beta2))
        
        # Check the second sumcheck relation:
        # a(β₂) - b(β₂)·(β₂·g₂(β₂) + t(β₁)/m) = h₂(β₂)·v_K(β₂)
        second_sumcheck = a_beta2 - b_beta2 * (beta_2 * g2_beta2 + t_beta1/m)
        expected_second_sumcheck = h2_beta2 * v_K_beta2
        
        if second_sumcheck != expected_second_sumcheck:
            print("❌ Second sumcheck relation failed")
            print(f"   Computed: {second_sumcheck}")
            print(f"   Expected: {expected_second_sumcheck}")
            return False
        
        # Verify KZG proofs
        # 1. Verify proof for evaluations at β₁
        beta1_commitments = [w_comm, zA_comm, zB_comm, zC_comm, h0_comm, s_comm, 
                             t_comm, g1_comm, h1_comm]
        if not self.kzg.check(rk, beta1_commitments, beta_1, evals_beta1, kzg_proof_beta1, xi_1):
            print("❌ KZG proof verification failed for β₁")
            return False
        
        # 2. Verify proof for all evaluations at β₂ (including indexer polynomials)
        beta2_commitments = [g2_comm, h2_comm] + index_commitments
        if not self.kzg.check(rk, beta2_commitments, beta_2, evals_beta2, kzg_proof_beta2, xi_2):
            print("❌ KZG proof verification failed for β₂")
            return False
        
        # All checks passed
        return True


# Generate the proof
print("\nGenerating proof...")
proof = prover.prove(ipk, x, w)

# Verify the proof
print("\nVerifying proof...")
result = verifier.verify(ivk, x, proof)

print(f"\n{'✅' if result else '❌'} Proof verification: {result}")


Generating proof...

Verifying proof...

✅ Proof verification: True


In [63]:
ivk

{'rk': ((5916915185941088365872896398876897552025493938136646107644345195313964161379, 17946651550764482016247062755986278429027985671174827982645894744920852758391),
  (10982256755009473410039842015658995586691617620270493277821505700781285565553, 15647338376027835987578459867294886528652942526045632187973014153857449985640),
  (19971282097381968079595681048798803037420347327508969557721177005736871948090, 5616392869431850967709185282078485980820771311062727848238602985217268932828)),
 'commitments': [(4843562528236702447463097725135703869344218612257982447535298863215099731008,
   10723740840451664408883923255565583913745005302541434496434185781545191593971,
   12412566717799148345334948864595258274738778598329700279063675412717622392764),
  (12292866647844542298699816189496289852963591607118671682883338332338503136050,
   4875453131714587303080959527584912454608313154184773956259929895088146338470,
   10833161428203264939576467997484822424164023111145394193657470353594821969160),
  

In [ ]:
from sage.all import load
from indexer import Indexer

print("Testing Marlin Verifier")
print("=" * 60)

# Load a test R1CS instance
RICS_INSTANCE = load("r1cs_instance.sobj")
A, B, C, z = RICS_INSTANCE["A"], RICS_INSTANCE["B"], RICS_INSTANCE["C"], RICS_INSTANCE["z"]

# Define public input (first few elements) and witness (remaining elements)
x_size = 5  # Adjust this based on your test instance
x = z[:x_size]
w = z[x_size:]

# Initialize the indexer, prover, and verifier
indexer = Indexer(curve_type="bn254")
prover = Prover(curve_type="bn254")
verifier = Verifier(curve_type="bn254")

# Determine maximum degree needed
max_degree = 200  # Adjust based on your specific instance

# Preprocess the constraint system
print("\nPreprocessing constraint system...")
ipk, ivk = indexer.preprocess(A, B, C, max_degree)

# Generate the proof
print("\nGenerating proof...")
proof = prover.prove(ipk, x, w)

# Verify the proof
print("\nVerifying proof...")
result = verifier.verify(ivk, x, proof)

print(f"\n{'✅' if result else '❌'} Proof verification: {result}")

if result:
    print("\nTesting tampered proof...")
    # Tamper with one of the evaluations
    proof["evaluations"]["beta1"][0] = (proof["evaluations"]["beta1"][0] + 1) % verifier.kzg.curve_order
    tampered_result = verifier.verify(ivk, x, proof)
    print(f"{'✅' if not tampered_result else '❌'} Tampered proof verification (should fail): {tampered_result}")

In [22]:
from indexer import Indexer

RICS_INSTANCE = load("r1cs_instance.sobj")
A, B, C, z = RICS_INSTANCE["A"], RICS_INSTANCE["B"], RICS_INSTANCE["C"], RICS_INSTANCE["z"]

# Initialize the indexer
indexer = Indexer(curve_type="bn254")

# Determine maximum degree needed
# In practice, we'd calculate this from the matrices and protocol requirements
max_degree = 200

print("\nPreprocessing constraint system...")
ipk, ivk = indexer.preprocess(A, B, C, max_degree)

ipk


Preprocessing constraint system...


{'ck': [(1, 2, 1),
  (...)],
 'A': [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [5 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [7 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0]
 [9 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0],
 'B': [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0

In [34]:
prover = Prover()

x,w = z[:5], z[5:]
prover.prove(ipk, x, w)

emang wazuh


In [46]:
from encoder import Encoder

encoder = Encoder(prover.kzg.curve_order)
encoder.update_state(A, B, C)

R = encoder.R
X = encoder.X
f = R.random_element(degree=12)

f

6702363821253605494981487882913328316774779141281018152507929354119565301386*X^12 + 8358985464456638550511053079018120569727461419632696427035713855621741421959*X^11 + 6425791482412372249648258601446900168954243136266681848167581347020678857036*X^10 + 4375865383934859412544119756855242132143337734943694003320572157084351591107*X^9 + 20502410395222650473426434166070502745665640980518497160424331522943000651622*X^8 + 6367618316742155626491913409008947536572602239481741800212839092713406957124*X^7 + 203906457716513351406541086174704734291752323576554309381858077357602121730*X^6 + 17629314232724164491533468503492371200405064338532685311565109542179617055728*X^5 + 15369709864908994898547985123556307016788083543679381131432617133887213528179*X^4 + 11003232230223645012999168949567085465994237123530922674050004032316866967174*X^3 + 1503133191732721322183687125713140782338280573018160577750374817921465050893*X^2 + 4413712022369382911753566052938261030713280937321752836544967204797812186225*X + 

In [50]:
g = X**5 + X**3 + 1

h = f // g

f == h*g

False

In [ ]:
Fq, H, X = encoder.Fq, encoder.H, encoder.X

v_H_x = prod([X - h for h in H[:x_size]])
v_H_w = prod([X - h for h in H[x_size:]])

w_poly = encoded_witness["w_poly"]
x_poly = encoded_witness["x_poly"]
zA_poly = encoded_combinations["zA_poly"]
zB_poly = encoded_combinations["zB_poly"]
zC_poly = encoded_combinations["zC_poly"]

# Add randomness for zero-knowledge (bounded independence)

# Random masking for w_poly, zA_poly, zB_poly, zC_poly
# These ensure zero-knowledge by adding random degree-b terms
b = 2
w_random = sum(Fq.random_element() * X**i for i in range(b))
zA_random = sum(Fq.random_element() * X**i for i in range(b))
zB_random = sum(Fq.random_element() * X**i for i in range(b))
zC_random = sum(Fq.random_element() * X**i for i in range(b))

w_masked = w_poly + w_random * v_H_w
z_masked = w_masked * v_H_x + x_poly

[1,
 14940766826517323942636479241147756311199852622225275649687664389641784935947,
 19540430494807482326159819597004422086093766032135589407132600596362845576832,
 7453743110195651009871841175551411207906567694170420694440975759997908783171,
 21888242871839275217838484774961031246007050428528088939761107053157389710902,
 15634706786522089014999940912207647497621112715300598509090847765194894752723,
 13274704216607947843011480449124596415239537050559949017414504948711435969894,
 20580681596408674675161806693190042586237586932987042748222592033583012763427,
 21888242871839275222246405745257275088548364400416034343698204186575808495616,
 6947476045321951279609926504109518777348511778190758694010539796934023559670,
 2347812377031792896086586148252853002454598368280444936565603590212962918785,
 14434499761643624212374564569705863880641796706245613649257228426577899712446,
 4407920970296243842541313971887945403937097133418418784715,
 625353608531718620724646483304962759092725168511543583460

In [164]:
Fq = encoder.Fq
R = encoder.R
X = encoder.X
H = encoder.H
K = encoder.K
v_H = encoder.v_H
R2 = PolynomialRing(R, 'Y')
Y = R2.gen()
alpha = 18077782573344409988674613190790018123865777734044923723881543902367686764352

row_A, col_A, val_A = encoded_matrices["row_A"], encoded_matrices["col_A"], encoded_matrices["val_A"]
z_poly = encoded_witness["z_poly"]
zA_poly = encoded_combinations["zA_poly"]

A_poly = sum((v_H(X)*v_H(Y)*val_A(kappa) // ((X - row_A(kappa))*(Y - col_A(kappa)))) for kappa in K)

r = R(encoder.u_H(alpha, X))
rA = sum(r(kappa)*A_poly(X=kappa, Y=X) for kappa in H)

s = r*zA_poly - rA*z_poly

s % v_H, sum(s(kappa) for kappa in H)

(18823425081190503679915573136164858072052702658456378893959586596410746533041*X^15 + 7043852157394625773054650232938441963697664769439181928878960122248723398581*X^14 + 14316296752383340033545047298039223521528399871914319613558498197748791739947*X^13 + 13576606242863977379406735506664935769876513685406054119192934390092642268434*X^12 + 20503542691138724978721448531353896791417787427878832398608510613533104059891*X^11 + 12756685917784172569792315230102840331541129524029465016808086017178579807672*X^10 + 16592412355645271998486117909506109660440383716661591374590387082310172179303*X^9 + 8451996821427446371864446532978591654059632547921351432162786971736300446802*X^8 + 8242237215029601628883075212323827242178876271202764947076014122880588894704*X^7 + 6767667217796345376977993854722046749951316923287380901025152687742663392203*X^6 + 13800418029452536201861522394087396945008141264315295507531102735427172668567*X^5 + 6104094015176309178152464769745639391879652540336824662288501908859023069

In [214]:
rA

19186660457895639171657332894686984142342154279041233208211979306788096277290*X^15 + 7665526952722085919681408175313623343188352366666031904095240895771551161305*X^14 + 12898452058055659975190095946032100316835621144724171926676708663580648434100*X^13 + 20599868983670140876090253819697047439036997409991528102550050749263043232257*X^12 + 19729325366191497755487447792951376417861805682780529226458875022366598566115*X^11 + 2153945764984075146596231743583352312279944248806846320003723282523120469224*X^10 + 11131186556952940625231970627337175102023065441884687075639138195464034149652*X^9 + 11564212677151868373159198481408916390179697679543055029579313754024455665720*X^8 + 9620077457880139744849184027835671922502362811568893394089372739356895059711*X^7 + 158044381274683654726715285397809135169129650064708534876382531056109476878*X^6 + 21017697245209472984238532785912814904221761936030468679606916646461229794610*X^5 + 127056206756634808488944021985627551924707242243533249228255245498236038252

In [216]:
alpha = 18077782573344409988674613190790018123865777734044923723881543902367686764352
rA(Y=alpha)

19186660457895639171657332894686984142342154279041233208211979306788096277290*X^15 + 7665526952722085919681408175313623343188352366666031904095240895771551161305*X^14 + 12898452058055659975190095946032100316835621144724171926676708663580648434100*X^13 + 20599868983670140876090253819697047439036997409991528102550050749263043232257*X^12 + 19729325366191497755487447792951376417861805682780529226458875022366598566115*X^11 + 2153945764984075146596231743583352312279944248806846320003723282523120469224*X^10 + 11131186556952940625231970627337175102023065441884687075639138195464034149652*X^9 + 11564212677151868373159198481408916390179697679543055029579313754024455665720*X^8 + 9620077457880139744849184027835671922502362811568893394089372739356895059711*X^7 + 158044381274683654726715285397809135169129650064708534876382531056109476878*X^6 + 21017697245209472984238532785912814904221761936030468679606916646461229794610*X^5 + 127056206756634808488944021985627551924707242243533249228255245498236038252

In [207]:
A_star = A.T

for i in range(A.ncols()):
    A_star[:, i] *= encoder.u_H(H[i], H[i])

A_star

[                                                                            0                                                                             0                                                                             0                                                                             0                                                                             0                                                                             0                                                                             0  8592011829774221812633976697329624974171832244859836928388079090326672383175 21888242871839275222246405745257275088548364400416034343698204186575808495505  1934524929899332937785641824363110103151849843104337632176819477382153561154                                                                             0 18797004331574745490858105341659180625507062160850682251865251157332377839178                                                                

In [211]:
A_star[0, 7], A[7, 0]*encoder.u_H(H[7], H[7])

(8592011829774221812633976697329624974171832244859836928388079090326672383175,
 8592011829774221812633976697329624974171832244859836928388079090326672383175)

In [212]:
encoder.update_state(A_star, B, C)

# Encode matrices
encoded_matrices = encoder.encode_matrices()

row_A, col_A, val_A = encoded_matrices["row_A"], encoded_matrices["col_A"], encoded_matrices["val_A"]
z_poly = encoded_witness["z_poly"]
zA_poly = encoded_combinations["zA_poly"]

A_poly_star = sum((v_H(X)*v_H(Y)*val_A(kappa) // ((X - row_A(kappa))*(Y - col_A(kappa)))) for kappa in K)

A_poly_star

(4429925664208591216842756218052866601473789820133550303981505753931634449794*X^15 + 3565699372517906142969018246227446662021016615712001106550682082616440810072*X^14 + 3034898395940662395607897684442170338083520817903424386658596099447465826904*X^13 + 4104045538469864103895706016592223838943986201835009851697344714144312918885*X^12 + 4912070105658084139845005826879183513106461463494991685672462911261888929479*X^11 + 12165398341845105256508191597441413924655042577216491509794889630560582096362*X^10 + 11620342535233638811001812286441765712081192403459114168178465306456920096215*X^9 + 20520227692349320520856005386178695395514091625390032197217066424914820464642*X^8 + 2412303932784117623685024270096674110451393741223517740291345287899286952766*X^7 + 18322543499321369079277387499029828426527347784704033237147522103959367685548*X^6 + 20223513354931502860502706870076265729617607550793683064403633798960707251190*X^5 + 177841973333694111183506997286650512496043781985810244920008594724314955767

In [213]:
A_poly_star(X=H[3], Y=H[1])

20920980406889608753353584833075720036972439478863865527609794447884731715040

In [199]:
A_poly_star(Y=alpha, X=34)

13036062631722271758170964428607457639911005459191353243650462998216046170470

In [215]:
alpha = 18077782573344409988674613190790018123865777734044923723881543902367686764352
rA(Y=alpha)

19186660457895639171657332894686984142342154279041233208211979306788096277290*X^15 + 7665526952722085919681408175313623343188352366666031904095240895771551161305*X^14 + 12898452058055659975190095946032100316835621144724171926676708663580648434100*X^13 + 20599868983670140876090253819697047439036997409991528102550050749263043232257*X^12 + 19729325366191497755487447792951376417861805682780529226458875022366598566115*X^11 + 2153945764984075146596231743583352312279944248806846320003723282523120469224*X^10 + 11131186556952940625231970627337175102023065441884687075639138195464034149652*X^9 + 11564212677151868373159198481408916390179697679543055029579313754024455665720*X^8 + 9620077457880139744849184027835671922502362811568893394089372739356895059711*X^7 + 158044381274683654726715285397809135169129650064708534876382531056109476878*X^6 + 21017697245209472984238532785912814904221761936030468679606916646461229794610*X^5 + 127056206756634808488944021985627551924707242243533249228255245498236038252

In [185]:
rA

19186660457895639171657332894686984142342154279041233208211979306788096277290*X^15 + 7665526952722085919681408175313623343188352366666031904095240895771551161305*X^14 + 12898452058055659975190095946032100316835621144724171926676708663580648434100*X^13 + 20599868983670140876090253819697047439036997409991528102550050749263043232257*X^12 + 19729325366191497755487447792951376417861805682780529226458875022366598566115*X^11 + 2153945764984075146596231743583352312279944248806846320003723282523120469224*X^10 + 11131186556952940625231970627337175102023065441884687075639138195464034149652*X^9 + 11564212677151868373159198481408916390179697679543055029579313754024455665720*X^8 + 9620077457880139744849184027835671922502362811568893394089372739356895059711*X^7 + 158044381274683654726715285397809135169129650064708534876382531056109476878*X^6 + 21017697245209472984238532785912814904221761936030468679606916646461229794610*X^5 + 127056206756634808488944021985627551924707242243533249228255245498236038252

In [60]:
dir(A[10, 11].parent())

['CartesianProduct',
 'Element',
 'Hom',
 '_FiniteField_prime_modn__char',
 '_Hom_',
 '_IntegerModRing_generic__order',
 '_QuotientRing_nc__I',
 '_QuotientRing_nc__R',
 '__bool__',
 '__cached_methods',
 '__call__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__fraction_field',
 '__ge__',
 '__gens_dict',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__ideal_monoid',
 '__init__',
 '__init_extra__',
 '__init_subclass__',
 '__interface',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__make_element_class__',
 '__module__',
 '__mul__',
 '__ne__',
 '__new__',
 '__pari__',
 '__polynomial_ring',
 '__pow__',
 '__pyx_vtable__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__richcmp__',
 '__rmul__',
 '__rpow__',
 '__rtruediv__',
 '__rxor__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__temporarily_change_names',
 '__truediv__',
 '__vector

In [163]:
(s % v_H).constant_coefficient() * 16

10181679000485721567355548917235236761789096121311133932448235957312998843862

In [142]:
kappa = H[3]
zA_poly(kappa), sum(A_poly(kappa, iota)*z_poly(iota) for iota in H)

(1336336, 1785475660315)

In [154]:
kappa = H[8]
[A_poly(X=kappa, Y=iota) for iota in H]

[7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [149]:
A

[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
[5 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[7 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0]
[9 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0]
[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1]
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]

In [ ]:
(A*z)[3], 

1336336

In [72]:
(X / 1).parent()

Univariate Polynomial Ring in X over Finite Field of size 21888242871839275222246405745257275088548364400416034343698204186575808495617 (using NTL)

In [27]:
encoder = Encoder(kzg)
F = encoder.Fq
A,B,C,z = R1CS_INSTANCE.values()
A_ = matrix(F, A)
B_ = matrix(F, B)
C_ = matrix(F, C)
z_ = vector(F, z)

Az = A_*z_
Bz = B_*z_
Cz = C_*z_

[azi * bzi for azi, bzi in zip(Az, Bz)] == list(Cz)

True

In [40]:
encoder = Encoder(kzg)
encoder.update_state(A_, B_, C_)
encoded_matrices = encoder.encode_matrices()

encoded_matrices

{'row_A': 15938820405245734543156359274481664988228183464366427843492731679858462823403*X^31 + 11550142209807530053124152363104331706981805145473777079335433805407219710386*X^30 + 11296097220654015719530637327925681727811184458112717415619635435215497422410*X^29 + 19903765638814113110082035548506265881750356695395649604525232561689362455008*X^28 + 15495284502864377361914642864889521005886772346227747758396902881622396200593*X^27 + 12874065432142412373831672030893286718257901349454906152787341095666750395449*X^26 + 19091901063058015792652563572228423215751535030106890599383706604852173738311*X^25 + 7299126514405732385881359928835548156352587251148356249301248672343789993148*X^24 + 9583798415594174264799341173560461576132169390151686734380989402766476016523*X^23 + 10693974958890589112689140681031525840809513955810570136986245199690160824393*X^22 + 5814889546297513113525449679614700828616188391058885560708768702100961981280*X^21 + 1061935882692769964079549287944904238097748920753255355935

In [9]:
import sys
sys.path.insert(0, '/mnt/d/Kuliah/ctf/research/kzg-snark')
from kzg import KZG

kzg = KZG()

kzg.curve_order

21888242871839275222246405745257275088548364400416034343698204186575808495617

In [1]:
from sage.all import *
R1CS_INSTANCE = load("r1cs_instance.sobj")

A, B, C, z = R1CS_INSTANCE['A'], R1CS_INSTANCE['B'], R1CS_INSTANCE['C'], R1CS_INSTANCE['z']

z

(1, 21888242871839275222246405745257275088548364400416034343698204186575808495583, 1785475660349, 1156, 21888242871839275222246405745257275088548364400416034343698204186575808456313, 1336336, 21888242871839275222246405745257275088548364400416034343698204186575763060193, 1544804416, 21888242871839275222246405745257275088548364400416034343698204186523285145473, 1785793904896, 21888242871839275222246405745257275088548364400416034343698204186575808299097, 21888242871839275222246405745257275088548364400416034343698204186575490447649, 21888242871839275222246405745257275088548364400416034343698204186575808495549, 21888242871839275222246405745257275088548364400416034343698204186575490251129, 21888242871839275222246405745257275088548364400416034343698204186575808495558, 21888242871839275222246405745257275088548364400416034343698204186575490251070)

In [4]:
from kzg import KZG

kzg

ModuleNotFoundError: No module named 'kzg'

In [55]:
z = [0,1,2,3,4,5,6,7,8,9]

x = z[:5]

len(x)

5

In [56]:
z[len(x):]

[5, 6, 7, 8, 9]